## Ground-Up CENSUS FOOD DESERT ANALYSIS

#### This module is to ignore potential unwanted warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd # manipulation of datasets  
import geopandas as gpd # manipulation of datasets with shapes/geometry
import matplotlib.pyplot as plt # visualization of the datasets
import os
import numpy as np
import requests 
import folium
from shapely.geometry import Point, box
import time

#### Importing Network Analysis Modules

In [ ]:
import osmnx as ox
import networkx as nx
from joblib import Parallel, delayed

### Reading State Census Tract File for a Year. (Choose Desired Year)

#### This file can be found at https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html#list-tab-790442341

In [ ]:
State = gpd.read_file("???.shp")

In [ ]:
State # see the table

### Download Poverty, Income, Population, Vehicle Access Datasets from data.census.gov

#### To compare with USDA use advanced filter from 2014-2018 since they have food desert data until 2019.
#### For latest data i.e., in 2025, use ACS 2019-2024 Average

In [ ]:
# poverty 2014-2018 ACS 
poverty = pd.read_csv("????.csv",skiprows=[1,1]) # we skip the 2nd row (since python indexing works as 0 , 1, 2 ....) as it a filler row

In [ ]:
# income 2014-2018 ACS
income = pd.read_csv("????.csv",skiprows=[1,1])

In [ ]:
# population 2019, derived from 2010 census
population = pd.read_csv("????.csv",skiprows=[1,1])

In [ ]:
# vehicle access 2014-2018 ACS
vehicle = pd.read_csv("????.csv",skiprows=[1,1])

In [ ]:
poverty['GEO_ID'] = poverty['GEO_ID'].str.split('S').str[1] # we do not change anything here since we fix the GEOID to enable joing with State Shape file
income['GEO_ID'] = income['GEO_ID'].str.split('S').str[1]
population['GEO_ID'] = population['GEO_ID'].str.split('S').str[1]
vehicle['GEO_ID'] = vehicle['GEO_ID'].str.split('S').str[1]

### Merge the Datasets with Census Tract for the State

In [ ]:
# Convert both columns of shape and census data to same data type (string in this case) to ensure proper merging
State["GEOID"] = State["GEOID"].astype(str)
poverty["GEO_ID"] = poverty["GEO_ID"].astype(str)
income["GEO_ID"] = income["GEO_ID"].astype(str)
population["GEO_ID"] = population["GEO_ID"].astype(str)
vehicle["GEO_ID"] = vehicle["GEO_ID"].astype(str)

In [ ]:
# Select specific columns from each DataFrame
# Adding GEO_ID for merging
# Rename columns from each DataFrame
# all of this should remain constant and should not change as data.census.gov follow a standard for assigning particular to variables. 

poverty_subset = poverty.iloc[:, [0, 2, 124, 126]].rename(columns={
    poverty.columns[2]: 'totalpop',
    poverty.columns[124]: 'belowpoverty',
    poverty.columns[126]: 'percentpoverty',
    poverty.columns[0]: 'GEO_ID'
})

income_subset = income.iloc[:, [0, 2, 24, 26, 34, 100, 106]].rename(columns={
    income.columns[2]: 'tothouseholds',
    income.columns[24]: 'medhouse',
    income.columns[26]: 'meanhouse',
    income.columns[34]: 'totfam',
    income.columns[100]: 'medfam',
    income.columns[106]: 'meanfam',
    income.columns[0]: 'GEO_ID'
})

population_subset = population.iloc[:, [0, 2, 172]].rename(columns={
    population.columns[2]: 'pop2010',
    population.columns[172]: 'houseunits',
    population.columns[0]: 'GEO_ID'
})

vehicle_subset = vehicle.iloc[:, [0, 2, 54, 78]].rename(columns={
    vehicle.columns[2]: 'occhousingunits',
    vehicle.columns[54]: 'novehicles',
    vehicle.columns[78]: 'percenthouseunits',
    vehicle.columns[0]: 'GEO_ID'
})

In [ ]:
# Merge all DataFrames to enable visualization
State_Food = (State
                     .merge(poverty_subset, left_on="GEOID", right_on="GEO_ID", how="inner")
                     .drop(columns="GEO_ID")  # Drop GEO_ID after every merge
                     .merge(income_subset, left_on="GEOID", right_on="GEO_ID", how="inner")
                     .drop(columns="GEO_ID")  
                     .merge(population_subset, left_on="GEOID", right_on="GEO_ID", how="inner")
                     .drop(columns="GEO_ID") 
                     .merge(vehicle_subset, left_on="GEOID", right_on="GEO_ID", how="inner")
                     .drop(columns="GEO_ID"))  

# If there are still duplicates, remove them
State_Food = State_Food.loc[:, ~State_Food.columns.duplicated()]

In [ ]:
State_Food # display the final file that we will be working

### Fix/Clean the Data

In [ ]:
# replace X with NaN if needed
State_Food = State_Food.replace('(X)', np.nan) # usually USDA uses (X) instead of NaN to represent no value so this should be changed 
                                                # to enable plotting

### Save the Final State Food Desert File

In [ ]:
State_Food.to_file("?????.shp")

### Dividing by Grid Cells (0.5 km)

In [ ]:
# Step 1: Define US bounds in lat/lon (WGS84, EPSG:4326)
us_bounds = {
    'minx': -125.0,  # Western edge (Washington)
    'miny': 24.396308,  # Southern edge (Florida)
    'maxx': -66.93457,  # Eastern edge (Maine)
    'maxy': 49.384358  # Northern edge (Washington/Minnesota)
}

# Create a bounding box for the US in lat/lon
us_bbox = gpd.GeoDataFrame(
    geometry=[box(us_bounds['minx'], us_bounds['miny'], us_bounds['maxx'], us_bounds['maxy'])],
    crs="EPSG:4326"
)

# Project to Albers Equal Area (EPSG:5070) for accurate grid cell size
us_bbox = us_bbox.to_crs(epsg=5070)
minx, miny, maxx, maxy = us_bbox.total_bounds

In [ ]:
# Define grid cell size (0.5 km = 500 meters)
cell_size = 500

# Create a grid covering the US
x_coords = np.arange(minx, maxx + cell_size, cell_size)
y_coords = np.arange(miny, maxy + cell_size, cell_size)
grid_cells = [box(x, y, x + cell_size, y + cell_size) for x in x_coords[:-1] for y in y_coords[:-1]]
grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=us_bbox.crs)

# Step 2: Load state census tracts and project to the same CRS
state_tracts = State_Food.copy()
state_tracts = state_tracts.to_crs(epsg=5070) # Albers Equal Area, this can be changed later

In [ ]:
# Clip the US grid to States boundaries, such that we are left with grids only in the State
state_bounds = state_tracts.dissolve().geometry[0]  # Combine all State tracts into one geometry
grid_state = gpd.sjoin(grid, state_tracts[['geometry']], how='inner', predicate='intersects')
grid_state = grid_indiana.drop(columns=['index_right'])

In [ ]:
# Step 3: Intersect grid with State tracts and calculate intersection areas, not every grid will hold the same weightage (the ones on the edge for ex)
grid_state = gpd.overlay(grid_state, state_tracts, how='intersection')
grid_state['area'] = grid_state.geometry.area  # Area of each intersection in square meters

In [ ]:
# Step 4: Assign population based on tract population (using 'pop2010')
# Calculate tract area and population density based on grids size
state_tracts['tract_area'] = state_tracts.geometry.area
state_tracts['pop_density'] = state_tracts['pop2010'].astype(float) / state_tracts['tract_area']

In [ ]:
# Merge population density into grid_indiana
grid_state = grid_indiana.merge(state_tracts[['GEOID', 'pop_density']], on='GEOID', how='left')

In [ ]:
# Calculate population for each grid cell portion
grid_state['grid_population'] = grid_state['area'] * grid_state['pop_density']

In [ ]:
# Final Step: Handle grid cells spanning multiple tracts by summing population contributions
grid_state['grid_id'] = grid_state.index  # Temporary unique ID
grid_population = grid_state.groupby('grid_id')['grid_population'].sum().reset_index()

In [ ]:
grid_state # this is what we need

In [ ]:
# save the grided file

In [ ]:
grid_state.to_file("?????.shp")

## Checkpoint

### Read the Grided State File

In [ ]:
grid_state = gpd.read_file("?????.shp") # read the shape file if needed

In [ ]:
grid_state.plot() # see how the map looks like, should take ~1 min to load since there a lot of grids

### Centroid Calculation

In [ ]:
# Step 1: Calculate centroids for each grid cell in grid_indiana
grid_state['centroid'] = grid_state.geometry.centroid # this is for our Network Analysis

## Get Grocery Locations through Open Street Map Geocoding

In [ ]:
# looking up when the store opens opened to filter by year??

In [ ]:
def get_locations(categories, queries, cities, state, country, brand=None):
    """Fetch locations from OpenStreetMap using Overpass API."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    all_locations = []

    # Construct the optional brand filter
    brand_filter = f'["brand"="{brand}"]' if brand else ''

    # Handle multiple categories, queries, and cities
    if not categories or not queries or not cities:
        print("No categories, queries, or cities provided.")
        return []

    for category in categories:
        for city in cities:
            for query in queries:
                overpass_query = f"""
                [out:json];
                area[name="{city}"]->.searchArea;
                (
                  node["{category}"="{query}"]{brand_filter}(area.searchArea);
                  way["{category}"="{query}"]{brand_filter}(area.searchArea);
                  relation["{category}"="{query}"]{brand_filter}(area.searchArea);
                );
                out center;
                """
                
                try:
                    response = requests.get(overpass_url, params={'data': overpass_query})
                    response.raise_for_status()
                    data = response.json()
                    all_locations.extend(data.get("elements", []))
                except requests.exceptions.RequestException as e:
                    print(f"Request error for {category}={query} in {city}: {e}")
                except requests.exceptions.JSONDecodeError:
                    print(f"Error decoding JSON response from API for {category}={query} in {city}.")

    return all_locations

In [ ]:
def plot_locations(data, city, state, country):
    """Plot locations on a Folium map."""
    if not data:
        print("No locations found.")
        return None

    # Extract the first valid location for map centering
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            m = folium.Map(location=[lat, lon], zoom_start=12)
            break
    else:
        print("No valid coordinates found.")
        return None

    # Add markers
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            folium.Marker([lat, lon], popup=f"{name} ({lat}, {lon})").add_to(m)

    return m

In [ ]:
def display_locations(data):
    """Display location names with coordinates in a DataFrame."""
    locations = []
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            locations.append([name, lat, lon])
    
    df = pd.DataFrame(locations, columns=['Name', 'Latitude', 'Longitude'])
    return df

In [ ]:
# Example Query Parameters
category = ["shop"]  # General category
queries = ["supermarket", "department_store", "greengrocer", "farm", "health_food", "retail"]  # Multiple specific queries for Food Desert
cities = ["????"]
state = "????"
country = "USA"
brand = None  # Change to "Walmart" or "Target" if needed or None 

# Fetch Data
data = get_locations(category, queries, cities, state, country, brand)

# Plot Data on Map
map_result = plot_locations(data, cities[0] if cities else None, state, country)

# Display DataFrame of Locations
df_locations = display_locations(data)

# Display the map and data
if map_result:
    display(map_result)

display(df_locations)

#### Convert fetched locations (df_locations) to a GeoDataFrame with Point geometries to perform distance calculation

In [ ]:
supermarkets = gpd.GeoDataFrame(
    df_locations,
    geometry=[Point(lon, lat) for lon, lat in zip(df_locations['Longitude'], df_locations['Latitude'])],
    crs="EPSG:4326")

In [ ]:
supermarkets

In [ ]:
supermarkets = supermarkets.to_crs(epsg=5070)  # Project to Albers Equal Area since that is our projection for now

In [ ]:
supermarkets_gdf = supermarkets

### We download the Road Network using Network Analysis Module with Python

In [ ]:
G = ox.graph_from_place("??????, USA", network_type="drive") # uncomment after downloading the road-network for the area

In [ ]:
#ox.save_graphml(G, filepath="??????.graphml") # save it such that you do not need to download it again

In [ ]:
#G = ox.load_graphml("?????.graphml")

In [ ]:
G = ox.project_graph(G, to_crs="EPSG:5070")

In [ ]:
ox.plot_graph(G) # plot the road network

#### Calculate the Nearest Supermarket from Grid

In [ ]:
# Step 4: Find nearest OSM nodes for grid centroids and supermarkets using vectorized approach
start_time = time.time()

# Extract x, y coordinates as NumPy arrays
grid_x = grid_state['centroid'].apply(lambda point: point.x).to_numpy()
grid_y = grid_state['centroid'].apply(lambda point: point.y).to_numpy()
market_x = supermarkets_gdf.geometry.x.to_numpy()
market_y = supermarkets_gdf.geometry.y.to_numpy()

# Use OSMnx vectorized nearest_nodes function
grid_state['nearest_node'] = ox.distance.nearest_nodes(G, grid_x, grid_y)
supermarkets_gdf['nearest_node'] = ox.distance.nearest_nodes(G, market_x, market_y)

end_time = time.time()
print(f"Nearest Nodes Execution Time: {end_time - start_time:.2f} seconds")

In [ ]:
# Convert to sets to avoid duplicates
grid_nodes = set(grid_indiana['nearest_node'].dropna())
market_nodes = set(supermarkets_gdf['nearest_node'].dropna())

In [ ]:
# Step 5: Compute shortest paths from all supermarket nodes to all grid nodes in one go
start_time = time.time()

# Use multi-source Dijkstra to find the shortest path from the closest supermarket to each node
path_lengths = nx.multi_source_dijkstra_path_length(G, sources=market_nodes, weight='length') # much faster the single source approach

# Extract distances for grid nodes only, convert to km
grid_to_market_distances = {
    grid_node: path_lengths.get(grid_node, float('inf')) / 1000  # Convert to km
    for grid_node in grid_nodes
}

# Map distances back to the DataFrame
grid_state['distance'] = grid_state['nearest_node'].map(grid_to_market_distances)

end_time = time.time()
print(f"Network Distance Execution Time: {end_time - start_time:.2f} seconds")

In [ ]:
grid_state['distance'] = grid_state['distance'] * 0.621371 # convert to miles as done by USDA

In [ ]:
grid_state['distance'].median() # check the mean and median if needed

In [ ]:
# Clean up: drop temporary columns if desired
grid_state = grid_state.drop(columns=['centroid', 'nearest_node']) # need to drop these in order to save the file

In [ ]:
grid_state.to_file("??????.shp") # you can save it if you like with distances calculated 

# Plotting Time!

In [ ]:
State = gpd.read_file("???????.shp")

In [ ]:
State.columns.tolist() # see column names to understand what we are working with

In [ ]:
State['GEOID'].describe() # just making sure the "unique" GEOID matches the total census tracts for the state

### Pick Any Census Tract within this Grided State to see how it looks like

In [ ]:
Tract = State[State['NAMELSAD'] == '??????']

In [ ]:
Tract['tothouseho'].describe()

In [ ]:
Tract['grid_households'].describe()

In [ ]:
Tract["area"].describe()

In [ ]:
Tract["grid_popul"].describe()

In [ ]:
Tract.plot(column='distance', legend=True, cmap='viridis') # distance
## Darker colored grids indicates there are supermarkets closer to the grids

### Read the pre-calculated State Median Income csv file (see other notebook on how to perform this!)

In [ ]:
# Load the metro income data
Metro = pd.read_csv("???????.csv")

In [ ]:
State["medfam"] = pd.to_numeric(State["medfam"], errors="coerce") # coverting the columns into numeric data type to ensure numpy calculations
State["percentpov"] = pd.to_numeric(State["percentpov"], errors="coerce")

### Plotting Low Income, Low Access, Low Income and Low Access 

In [ ]:
# Drop low_income_status if it already exists to avoid merge conflicts
if 'low_income_status' in State.columns:
    State = State.drop(columns=['low_income_status'])

In [ ]:
# Step 1: Compute low-income status at tract level
medfam_threshold = 0.8 * State["medfam"].median()

def get_metro_threshold(row, metro_df):
    if row["Metro"] == "Yes":
        metro_row = metro_df[metro_df["Name"] == row["NAME_1"]]
        if not metro_row.empty:
            return 0.8 * metro_row["Median_Income"].iloc[0]
    return None

def is_low_income(row, metro_df):
    metro_threshold = get_metro_threshold(row, metro_df)
    if row["percentpov"] >= 20 or row["medfam"] <= medfam_threshold or (metro_threshold is not None and row["medfam"] <= metro_threshold):
        return "Yes"
    return "No"

State['GEOID'] = State['GEOID'].astype(str)
tract_low_income = State.groupby('GEOID').first().reset_index()
tract_low_income['GEOID'] = tract_low_income['GEOID'].astype(str)
tract_low_income['low_income_status'] = tract_low_income.apply(lambda row: is_low_income(row, Metro), axis=1)

State = State.merge(tract_low_income[['GEOID', 'low_income_status']], on='GEOID', how='left')

In [ ]:
# Step 2: Estimate households per grid
tract_area_totals = State.groupby('GEOID')['area'].sum().reset_index()
tract_area_totals.rename(columns={'area': 'total_area'}, inplace=True)
State = State.merge(tract_area_totals, on='GEOID', how='left')
State['area_proportion'] = State['area'] / State['total_area']
State['grid_households'] = State['tothouseho'] * State['area_proportion']
State['grid_novehicles'] = State.apply(
    lambda row: (row['novehicles'] / row['tothouseho'] * row['grid_households']) if row['tothouseho'] > 0 and row['novehicles'] > 0 else 0,
    axis=1
)

In [ ]:
# Step 3: Create a dictionary of unique GEOIDs
tract_dict = {}
for geoid in State['GEOID'].unique():
    tract_dict[geoid] = State[State['GEOID'] == geoid][[
        'low_income_status', 'grid_popul', 'distance', 'totalpop', 'geometry',
        'grid_households', 'grid_novehicles'
    ]]

In [ ]:
# Step 4: Define urban/rural classification and distance thresholds
def is_rural(row):
    return row['totalpop'] < 2500

urban_distance_threshold = 1.0 # miles
rural_distance_threshold = 10.0  # miles
no_vehicle_distance_threshold = 0.5  # miles
far_distance_threshold = 20.0  # miles

In [ ]:
# Step 5: Process tracts to evaluate low-income and low-access
all_tracts = []
for geoid, tract_data in tract_dict.items():
    tract_totalpop = tract_data['totalpop'].iloc[0]
    low_income_status = tract_data['low_income_status'].iloc[0]
    tract_geometry = tract_data['geometry'].unary_union
    
    # Low-income check
    is_low_income = low_income_status == "Yes"
    
    # Low-access check for low-income tracts
    is_low_access_low_income = False
    if is_low_income:
        is_tract_rural = is_rural(tract_data.iloc[0])
        distance_threshold = rural_distance_threshold if is_tract_rural else urban_distance_threshold
        far_grids = tract_data[tract_data['distance'] > distance_threshold]
        far_population = far_grids['grid_popul'].sum()
        if tract_totalpop > 0:
            far_percentage = (far_population / tract_totalpop) * 100
        else:
            far_percentage = 0
        if far_population >= 500 or far_percentage >= 33:
            is_low_access_low_income = True
    
    # Low-access check for non-low-income tracts (USDA criteria)
    is_low_access_non_low_income = False
    if not is_low_income:
        # Option 1: Households > 0.5 miles with no vehicle access
        far_no_vehicle_grids = tract_data[
            (tract_data['distance'] > no_vehicle_distance_threshold) &
            (tract_data['grid_novehicles'] > 0)
        ]
        far_no_vehicle_households = far_no_vehicle_grids['grid_novehicles'].sum()
        # Option 2: Population > 20 miles (no vehicle check)
        far_grids = tract_data[tract_data['distance'] > far_distance_threshold]
        far_population = far_grids['grid_popul'].sum()
        if tract_totalpop > 0:
            far_percentage = (far_population / tract_totalpop) * 100
        else:
            far_percentage = 0
        if far_no_vehicle_households >= 100 or far_population >= 500 or far_percentage >= 33:
            is_low_access_non_low_income = True
    
    # Low-access overall (either income-based or USDA criteria)
    is_low_access = is_low_access_low_income or is_low_access_non_low_income
    
    all_tracts.append({
        'GEOID': geoid,
        'low_income_status': low_income_status,
        'is_low_access': is_low_access,
        'is_low_access_low_income': is_low_access_low_income,
        'is_low_access_non_low_income': is_low_access_non_low_income,
        'geometry': tract_geometry
    })

all_tracts_gdf = gpd.GeoDataFrame(all_tracts, geometry='geometry')

In [ ]:
# Step 6: Reproject to Mercator (EPSG:3857)
all_tracts_gdf = all_tracts_gdf.set_crs(epsg=5070, allow_override=True)
all_tracts_gdf = all_tracts_gdf.to_crs(epsg=3857)

State_boundary = State.dissolve().boundary
State_boundary = State_boundary.set_crs(epsg=5070, allow_override=True)
State_boundary = State_boundary.to_crs(epsg=3857)

In [ ]:
# Step 7: Create three subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8), sharey=True)

# Plot 1: Low Income Only
State_boundary.plot(ax=ax1, color='none', edgecolor='gray', linewidth=1.0, alpha=0.7, zorder=1)
colors1 = ['green' if row['low_income_status'] == "Yes" else 'white' for _, row in all_tracts_gdf.iterrows()]
all_tracts_gdf.plot(color=colors1, edgecolor='black', linewidth=0.5, ax=ax1, alpha=0.5, zorder=2)
ax1.set_title('Low Income Only in State', fontsize=10)
legend_elements1 = [
    Patch(facecolor='green', edgecolor='black', label='Low-Income'),
    Patch(facecolor='white', edgecolor='black', label='Other Tracts', alpha=0.5)
]
ax1.legend(handles=legend_elements1, loc='upper right', title='Tract Status')
ax1.set_axis_off()

# Plot 2: Low Access Only
State_boundary.plot(ax=ax2, color='none', edgecolor='gray', linewidth=1.0, alpha=0.7, zorder=1)
colors2 = ['blue' if row['is_low_access'] else 'white' for _, row in all_tracts_gdf.iterrows()]
all_tracts_gdf.plot(color=colors2, edgecolor='black', linewidth=0.5, ax=ax2, alpha=0.5, zorder=2)
ax2.set_title('Low Access Only in State', fontsize=10)
legend_elements2 = [
    Patch(facecolor='blue', edgecolor='black', label='Low-Access'),
    Patch(facecolor='white', edgecolor='black', label='Other Tracts', alpha=0.5)
]
ax2.legend(handles=legend_elements2, loc='upper right', title='Tract Status')
ax2.set_axis_off()

# Plot 3: Low Income and Low Access
State_boundary.plot(ax=ax3, color='none', edgecolor='gray', linewidth=1.0, alpha=0.7, zorder=1)
colors3 = ['red' if row['low_income_status'] == "Yes" and row['is_low_access'] else 'white' for _, row in all_tracts_gdf.iterrows()]
all_tracts_gdf.plot(color=colors3, edgecolor='black', linewidth=0.5, ax=ax3, alpha=0.5, zorder=2)
ax3.set_title('Low Income and Low Access in State', fontsize=10)
legend_elements3 = [
    Patch(facecolor='red', edgecolor='black', label='Low-Income & Low-Access'),
    Patch(facecolor='white', edgecolor='black', label='Other Tracts', alpha=0.5)
]
ax3.legend(handles=legend_elements3, loc='upper right', title='Tract Status')
ax3.set_axis_off()

# Adjust layout to prevent overlap
plt.tight_layout()